<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

In [15]:
from azure.identity import AzureCliCredential
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.sql import SqlManagementClient
from azure.mgmt.sql.models import FirewallRule

import pyodbc

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

In [2]:
credential = AzureCliCredential()
sql_client = SqlManagementClient(credential, AZURE_SUBSCRIPTION_ID)

In [3]:
sql_server = sql_client.servers.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    'bens-server',
    {
        'location': DEFAULT_LOCATION,
        'version': '12.0',
        'administrator_login': 'benadmin',
        'administrator_login_password': 'testpassword123!'
    }
)
sql_server.wait()

In [4]:
for server in sql_client.servers.list():
    print(server.name)

bens-server


In [5]:
database = sql_client.databases.begin_create_or_update(
    DEFAULT_RESOURCE_GROUP,
    'bens-server',
    'bens-db',
    {
        'location': DEFAULT_LOCATION,
        'sample_name': 'AdventureWorksLT',
        'edition': 'GeneralPurpose',
        'compute_model': 'Serverless',
        'family': 'Gen5',
        'capacity': 2
    }
)
database.wait()

In [6]:
for db in sql_client.databases.list_by_server(DEFAULT_RESOURCE_GROUP, 'bens-server'):
    print(db.name)

master
bens-db


In [13]:
firewall = sql_client.firewall_rules.create_or_update(
    DEFAULT_RESOURCE_GROUP,
    'bens-server',
    'AllowMyIp',
    {
        'start_ip_address': '192.168.1.39',
        'end_ip_address': '192.168.1.39'
    }
)

In [14]:
for fr in sql_client.firewall_rules.list_by_server(DEFAULT_RESOURCE_GROUP, 'bens-server'):
    print(fr.name)

AllowMyIp


In [24]:
SERVER = 'bens-server.database.windows.net'
DATABASE = 'bens-db'
USERNAME = 'benadmin'
PASSWORD = 'testpassword123!'
# Has to be installed
DRIVER = '{ODBC Driver 18 for SQL Server}'

connection_string = f'DRIVER={DRIVER};PORT=1433;SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

In [26]:
firewall = sql_client.firewall_rules.create_or_update(
    DEFAULT_RESOURCE_GROUP,
    'bens-server',
    'JupyterNotebook',
    {
        'start_ip_address': '72.175.52.98',
        'end_ip_address': '72.175.52.98'
    }
)


In [27]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()